<a href="https://colab.research.google.com/github/idiyak/PhysicsProjects/blob/main/fishpythonscriptdocumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fish Python Script Documentation**
**Documentation by:** Idil Yaktubay (iyaktubay@iisd-ela.org)

## **1. About This Document**
&nbsp; &nbsp; &nbsp; &nbsp;This file documents the `FishPythonScript.py` script to explain the algorithms and code used to process fish data. The author of the script is Tonja Dwyer (tonjadwyer@gmail.com), though some comments in the script were edited/added by me. The script is stored in the PythonCaller transformer within the FME Workbench named 'wb4 - fish id finder', which assigns fish individual IDs to observations - either new IDs or existing ones from the database. However, the code in this document is fully runnable on its own, as long as the instructions in section 2 are followed. Notice that in the PythonCaller version of the script, the modules `fme` and `fmeobjects` are imported for the transformer to work. Though, since this document is not connected to FME, importing these modules is not necessary to run the code in this document. Lastly, if you are accessing the GitHub copy of this file, click the "Open in Colab" link at the top of this document. This will take you to the interactive python notebook environment where you can run the code!

## **2. Importing the Example Input Data**
&nbsp; &nbsp; &nbsp; &nbsp;To document the `FishPythonScript.py` script, we use the input table `FishPythonInputTable.csv` (from https://github.com/IISD-ELA/FishDataProcessing) as an example case. The reader must upload this table to the files tab on the left panel before running the following block of code. Note that the purpose of the code in this section is to simply import the input table in the same structure as it would be imported on FME, so the code contents are irrelevant. Run the following code after uploading the example data:

In [ ]:
##RUN THIS CODE, BUT IGNORE ITS CONTENTS, NOT RELEVANT##
import csv

# Initialize an empty list to store the pairs
pairs_list = []

# Specify the CSV file name
csv_file = 'FishPythonInputTable.csv'

try:
    # Open the CSV file for reading
    with open(csv_file, 'r') as file:
        # Create a CSV reader object
        csv_reader = csv.reader(file)

        # Read the header row to determine column indices
        header = next(csv_reader)
        obs_number_index = header.index('obs_number')
        tag_id_index = header.index('tag_id')

        # Iterate through the rows and store the desired columns in the list
        for row in csv_reader:
            obs_number = row[obs_number_index]
            tag_id = row[tag_id_index]
            pairs_list.append([obs_number, tag_id])

except FileNotFoundError:
    print(f"The file '{csv_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

##**3. Brief Definition of Python Classes**

&nbsp; &nbsp; &nbsp; &nbsp;In Python, a `class` is a data type defined by the user. A `class` contains objects with the user-defined data type, as well as methods that can be used to manipulate these objects. `FishPythonScript` defines two classes, `UnionFind` and `FeatureProcessor`, which are documented in the following sections.

##**4. Understanding the Union-Find Algorithm**

&nbsp; &nbsp; &nbsp; &nbsp; The first class defined in the script is `UnionFind`, which allows the user to create and manipulate objects with data type `UnionFind` using the methods `__init__`, `find`, and `union`. Before the goal of `UnionFind` is explained, a couple of definitions:
* `parent`: An object's `parent` is the representative of the group that the object is in. For example, let `4` be the representative of group `a = {2, 8, 4, 9}`. Then, the `parent` of object `2` is `4` (likewise for `8`, `4`, and `9`).
***Join**: Joining two groups of objects is equivalent to making the parent of one group the parent of the other. For example, joining sets `a = {2, 8, 4, 9}`, with parent `4`, and `b = {1, 2}`, with parent `2`, may give us the set `b = {1, 2, 8, 4, 9}`, with parent `4` (or parent `2`, depending on which original parent we choose).

&nbsp; &nbsp; &nbsp; &nbsp; The purpose of creating the `UnionFind` algorithm is to first, find the `parent` of an object using the `find` method, then, to join two groups of objects using the `union` method. Let's run and examine the code:

In [ ]:
# Define the UnionFind data type/structure
class UnionFind:

    def __init__(self): # Construct a UnionFind object
        self.parent = {} # Initialize parent attribute as empty dictionary

    def find(self, u): # Find the parent of the group object u is in (find the parent of u)
        if u != self.parent.setdefault(u, u): # Check if u is its own parent
            self.parent[u] = self.find(self.parent[u]) # If u is not its own parent, keep searching for the parent
        return self.parent[u] # Output the parent of the group object u is in

    def union(self, u, v): # Joins the two groups that objects u and v are in
        u = self.find(u) # Find the parent of the group object u is in
        v = self.find(v) # Find the parent of the group object v is in
        if u != v: # Check if u and v have the same parent
            self.parent[u] = v # If not, make the parent of u the parent of v, thus joining the two groups

### **`__init__`**
&nbsp; &nbsp; &nbsp; &nbsp; Let's unpack the code. First, the `__init__` constructor method is called when the user first creates a new `UnionFind` object. The `parent` attribute of this new object gives us a dictionary, whose keys are the members of a group and whose values are the parents of these members. The following example illustrates this.

&nbsp; &nbsp; &nbsp; &nbsp; Let's create an example `UnionFind` object called `example_object`:

In [ ]:
example_object = UnionFind() # Create new object with type UnionFind

Because `example_object` has just been created, the `parent` attribute of `example_object` should be an empty dictionary, since the object contains no data. Run the following code and confirm that this is true:

In [ ]:
print('example_object is an empty dictionary:', example_object.parent)

example_object is an empty dictionary: {}


###**`find`**
&nbsp; &nbsp; &nbsp; &nbsp; When we call the `find` method for the first time, however, we will get a value regardless of the fact that the dictionary was just empty. See for yourself:

In [ ]:
example_member = 3 # Set an example member whose parent we are trying to find
example_object.find(example_member) # Find the parent of example_member

3

As we observe, the value returned as the parent of `example_member` is itself! This is because when the `find` method is called, it first checks if `example_member` is its own parent with `setdefault`. When `setdefault(key, value)` is called, it looks up the value of the key `key`. If `key` does not exist, it creates a key `key` and assigns it the value `value`. In our case, since the dictionary `example_object.parent` was empty, `setdefault` created a new key `example_member` and assigned it the value `example_member`, initializing `example_member` as its own parent. Then, the `if` statement evaluated to `FALSE`, skipped the line directly below, and returned `example_object.parent[example_member]`, which is `example_member` itself - thanks to `setdefault`. Run and confirm:

In [ ]:
example_object.parent[example_member]

3

###**`union`**
&nbsp; &nbsp; &nbsp; &nbsp; The third method of `UnionFind` is `union`, which joins the two groups of any two members, given that they are not already in the same group. Let's closely inspect what happens when we run the following code:

In [ ]:
example_object.union(1, 2) #1
example_object.union(2, 5) #2
example_object.union(3, 5) #3
example_object.union(1, 5) #4
example_object.union(5, 7) #5
print('The parents of 1, 2, 3, 5, 7 are', example_object.find(1), ',', example_object.find(2), ',', example_object.find(3), ',', example_object.find(5), ',',  example_object.find(7), ',', 'respectively.') #6

The parents of 1, 2, 3, 5, 7 are 7 , 7 , 7 , 7 , 7 , respectively.


The output suggests that the elements `1`, `2`, `3`, `5`, and `7` all belong to the same group `{1, 2, 3, 5, 7}` because they all have the same parent `7`. To understand how this output was produced, we now follow what happens in the first two lines of the code (the remaining lines aren't explained as the same process repeats each time):

- `example_object.union(1, 2)`
  - First, the program executes the first line of the method `union`, which is `u = example_object.find(1)`. At this point, as the key `1` does not yet exist in `example_object.parent`, `1` is initialized as its own parent (for the same reasons outlined in the `find` section), and thus the variable `u` is assigned the value `1`. Same thing happenes with variable `v`, and thus it is assigned the value `2`. Then, the `if` statement evaluates to `TRUE` and thus makes `v` the parent of `1`. Therefore, the parent of `1` is now `2`, and the group that `1` was initially in is now joined with the group that `2` is in.
- `example_object.union(2, 5)`
  - As before, the program assigns the variables `u` and `v` the values `2` and `5`, respectively (because `2` is already its own parent at this point, and `5` is initialized as its own parent). Once again, the `if` statement evaluates to `TRUE` and makes `v` the parent of `2`. Therefore, the parent of `2` is now `5`, and the group that `2` and `1` were initially in is joined with the group that `5` is in. However, note that until the `find` method is called for `1` (i.e., `example_object.find(1)`), its parent still remains as `2` in the `example_object.parent` dictionary. This is because the `find` method "flattens" the data structure to be more efficient everytime the user calls it.

The process outlined above might be a bit difficult to visualize, so every line of code in `[8]` is explained visually at: https://github.com/IISD-ELA/FishDataProcessing/blob/main/diagramUnionFind.png. Note that circles refer to parents and squares refer to children.

##**5. FeatureProcessor: Applying the UnionFind Algorithm to Fish**

&nbsp; &nbsp; &nbsp; &nbsp; The second class defined in the script is `FeatureProcessor`, which takes an input data table from the FME workbench, and outputs a data table that assigns a unique group ID to each group of observation numbers that have the same parent (the observations for the same fish). The following block of code defines the `FeatureProcessor` class, with comments added to explain everything line by line - though each relevant part will be discussed in detail in the following sections.

In [ ]:
# FeatreProcessor class runs when the python caller wakes up
from collections import defaultdict
class FeatureProcessor(object):
    """Template Class Interface:
    When using this class, make sure its name is set as the value of the 'Class
    to Process Features' transformer parameter.
    """

    def __init__(self):
        """Base constructor for class members."""

        # Initialize data attribute as an empty list
          # Will eventually be used to store lists consisting of two elements,
          # obs_number and tag_id
        self.data = []

        # Initialize uf attribute as a UnionFind object
        self.uf = UnionFind()

        # Initialize a dictionary to hold tags to obs_num mapping
         #  (we need to do this so we can use "UnionFind")
         #  (also note that here the default dictionary is a "set", which means it only holds one of each unique value - we don't want duplicates)
        self.tag_to_obs = defaultdict(set)

        # initialize set to hold distinct observation numbers (distinct, hence set!)
        self.distinct_obs = set() # Not sure why this is here, does not change anything about the program whatsoever


    # For FME Python Caller, automatically each feature that comes in will go through this "input" function
    def input(self, feature):
        """This method is called for each FME Feature entering the
        PythonCaller. If knowledge of all input Features is not required for
        processing, then the processed Feature can be emitted from this method
        through self.pyoutput(). Otherwise, the input FME Feature should be
        cached to a list class member and processed in process_group() when
        'Group by' attributes(s) are specified, or the close() method.

        :param fmeobjects.FMEFeature feature: FME Feature entering the
            transformer.
        """
        # From the input rows, we want to pull out these two specific values
        #  (we don't care about the other values coming in - we will join them back in after the Python script)
        obs_number = feature.getAttribute('obs_number')
        tag_id = feature.getAttribute('tag_id')

        # Store pairs of observation numbers and tag IDs in the self.data list
        self.data.append([obs_number, tag_id])


    def close(self):
        """This method is called once all the FME Features have been processed
        from input().
        """
        pass


    def process_group(self):
        """When 'Group By' attribute(s) are specified, this method is called
        once all the FME Features in a current group have been sent to input().

        FME Features sent to input() should generally be cached for group-by
        processing in this method when knowledge of all Features is required.
        The resulting Feature(s) from the group-by processing should be emitted
        through self.pyoutput().

        FME will continue calling input() a number of times followed
        by process_group() for each 'Group By' attribute, so this
        implementation should reset any class members for the next group.
        """

        # example tag_to_obs
        #dict_keys(['tag1', 'tag2', 'tag3', 'tag4', 'tag5'])
        #dict_values([{1, 2, 7}, {1, 2, 3}, {3, 4}, {5, 6}, {6}])

        # example distinct_obs
        #{1, 2, 3, 4, 5, 6, 7}

        # Step 1: Build tag to obs_num mapping
        # for each tag, which obs does it appear in?
        for obs, tag in self.data:
            self.tag_to_obs[tag].add(obs)

        # Step 2: Union obs with the same tag
        for obs in self.tag_to_obs.values(): # Iterate over the group of observations for every tag ID
            obs = list(obs) # For the current tag ID, store the corresponding group of observations in a list, for the purposes of indexing
            for i in range(1, len(obs)): # For every observation in the obs list, assign the observation after it to be its parent
                self.uf.union(obs[0], obs[i]) # Use the UnionFind class to accomplish the join

        # Step 3: Get the distinct obs
        distinct_obs = set() # Creates an empty set (a data structure with a unique constraint) where obs values will be accumulated
        for obs, tag in self.data: # Go through every observation number and fill distinct_obs will distinct obs values
            distinct_obs.add(obs)

        # Step 4: form the groups
        groups = defaultdict(set) # Empty dictionary with unique contraint where the groups from the final output file will be stored
                                  # This dictionary has parents as keys and group members as values
        for obs in distinct_obs: # Iterate over every distinct obs
            root = self.uf.find(obs) # Find the parent of obs, and thus flatten the structure created by step 2
            groups[root].add(obs) # Add the ultimate parent (parent after flattening) to groups dictionary

        feature = fmeobjects.FMEFeature()
        # Output the groups
        group_num = 1
        for _, members in groups.items():
            print(f"Group {group_num}: {sorted(list(members))}")
            group_num += 1
            members_as_string = [str(x) for x in members]
            group_members = "/".join(members_as_string)

            #fmeobjects.FMELogFile().logMessageString('GROUP: {} COUNT{}!!!!'.format(group_id, len(members)), fmeobjects.FME_INFORM)
            new_feature = feature.clone()

            new_feature.setAttribute('group_id', group_num)
            new_feature.setAttribute('group_members', group_members)


            self.pyoutput(new_feature)


    def has_support_for(self, support_type):
        """This method returns whether this PythonCaller supports a certain type.
        The only supported type is fmeobjects.FME_SUPPORT_FEATURE_TABLE_SHIM.

        :param int support_type: The support type being queried.
        :returns: True if the passed in support type is supported.
        :rtype: bool
        """
        if support_type == fmeobjects.FME_SUPPORT_FEATURE_TABLE_SHIM:
            # If this is set to return True, FME will pass features to the input() method that
            # come from a feature table object. This allows for significant performance gains
            # when processing large numbers of features.
            # To enable this, the following conditions must be met:
            #   1) features passed into the input() method cannot be copied or cached for later use
            #   2) features cannot be read or modified after being passed to self.pyoutput()
            #   3) Group Processing must not be enabled
            # Violations will cause undefined behavior.
            return False

        return False

###**`__init__`**

&nbsp; &nbsp; &nbsp; &nbsp; The first attribute defined by the `__init__` constructor is `data`, which initializes `self.data` as an empty `list`. When the later lines are executed, `self.data` will eventually fill with lists of `obs_number` and `tag_id` pairs, taking on the nested form `[[obs_number1, tag_id1], [obs_number2, tag_id2], ...]`. The second attribute is `uf`, which creates the `UnionFind` object `self.uf`. This is the attribute the script will use to implement the `UnionFind` algorithm to observation numbers. The third attribute is `tag_to_obs`, which will eventually hold `tag_id` values as keys and collections of `obs_number` values as values.

###**`input`**

&nbsp; &nbsp; &nbsp; &nbsp; The `input` method takes the input table from FME, extracts the values for the obs_number and tag_id columns, and assigns those values to the variables `obs_number` and `tag_id`. It then fills up the `self.data` list with lists of each observation-tag pair. Since this document is not connected to the FME workbench, we instead loaded the `FishPythonOutputTable.csv` file to see what `self.data` will look like after the `input` method has been called for an example input table (section 1). Run the following code, examine its output (which is what `self.data` will look like on FME when given the same input data table):

In [ ]:
FP_example = FeatureProcessor() # Create FeatureProcessor object
FP_example.data = pairs_list # Set the object's attribute to the list created back in section 1

print(FP_example.data)

[['12', '4000000000'], ['16', '4400000000'], ['20', '1110000000'], ['6', '8888888888'], ['7', '9999999999'], ['11', '3000000000'], ['13', '4000000000'], ['15', '1100000000'], ['18', '8800000000'], ['22', '4440000000'], ['23', '8880000000'], ['1', '111111111111111'], ['2', '5555555555'], ['3', '6666666666'], ['4', '7777777777'], ['5', '7777777777'], ['8', '1000000000'], ['9', '2000000000'], ['10', '2000000000'], ['14', '7000000000'], ['17', '7700000000'], ['19', '9900000000'], ['21', '3330000000'], ['25', '3330XX'], ['28', '7770XX'], ['29', '9990XX'], ['34', '3000XX'], ['37', '6000XX'], ['38', '1110YY'], ['33', '2000XX'], ['35', '3000XX'], ['40', '1110FF'], ['24', '1110XX'], ['26', '5550XX'], ['27', '5550XX'], ['30', '1000XX'], ['31', '1000XX'], ['32', '1000XX'], ['36', '4000XX'], ['39', '1110BB'], ['46', '11CX'], ['50', '11FX'], ['52', '11HX'], ['45', '11AX'], ['47', '11CX'], ['51', '11GX'], ['53', '11AY'], ['41', '11AX'], ['42', '11AX'], ['43', '11BX'], ['44', '11BX'], ['48', '11DX'],

###**`process_group`**

&nbsp; &nbsp; &nbsp; &nbsp;  In our example, step 1 creates a dictionary whose keys are `tag_id` and whose values are `obs_number` from the `FP_example.data` list. Each `tag_id` key is mapped to the group of observations that the tag has appeared in. Run the following code and examine the output:

In [ ]:
# Step 1: Build tag to obs_num mapping
# for each tag, which obs does it appear in?
for obs, tag in FP_example.data:
  FP_example.tag_to_obs[tag].add(obs)
print('The FP_example.tags_to_obs dictionary looks like: \n', FP_example.tag_to_obs)

The FP_example.tags_to_obs dictionary looks like: 
 defaultdict(<class 'set'>, {'4000000000': {'13', '12'}, '4400000000': {'16', '67'}, '1110000000': {'20'}, '8888888888': {'60', '6'}, '9999999999': {'7', '61'}, '3000000000': {'11'}, '1100000000': {'15', '65'}, '8800000000': {'18'}, '4440000000': {'22', '74'}, '8880000000': {'82', '23'}, '111111111111111': {'1'}, '5555555555': {'57', '2'}, '6666666666': {'3', '58'}, '7777777777': {'59', '5', '4'}, '1000000000': {'8'}, '2000000000': {'10', '9'}, '7000000000': {'14', '63'}, '7700000000': {'17'}, '9900000000': {'19'}, '3330000000': {'75', '21'}, '3330XX': {'56', '25'}, '7770XX': {'60', '28'}, '9990XX': {'61', '29'}, '3000XX': {'35', '34'}, '6000XX': {'64', '37'}, '1110YY': {'38', '66'}, '2000XX': {'33'}, '1110FF': {'81', '40'}, '1110XX': {'24'}, '5550XX': {'27', '59', '26', '58'}, '1000XX': {'31', '30', '32'}, '4000XX': {'62', '36'}, '1110BB': {'39', '79'}, '11CX': {'47', '46'}, '11FX': {'50', '72'}, '11HX': {'73', '52'}, '11AX': {'41', '

&nbsp; &nbsp; &nbsp; &nbsp;Next, step 2 creates a `UnionFind` dictionary in the form of `FP_example.uf.parent`. This dictionary maps every `obs_num` contained in the keys of `FP_example.tag_to_obs` to its parent. In creating (or simply filling, since it already existed but was empty) `FP_example.uf.parent`, the code below uses the `union` method from `UnionFind` to assign the next observation number in the group to be the parent of the one before it. For example, let `obs1 = {1, 4, 3, 12}` be a list during one of the iterations in the loop. At the end of the loop, `1` will have parent `4`, `4` will have parent `3`, `3` will have parent `12`, and `12` will be its own parent (assuming that the same observation number does not appear in another tag later on, which is possible since fish can have multiple tags, but the below code takes care of this as well!). Therefore, the numbers are not yet joined. Run the following code and examine the output:

In [ ]:
# Step 2: Union obs with the same tag
for obs in FP_example.tag_to_obs.values(): # Iterate over the group of observations for every tag ID
  obs = list(obs) # For the current tag ID, store the corresponding group of observations in a list, for the purposes of indexing
  for i in range(1, len(obs)): # For every observation in the obs list, assign the observation after it to be its parent
    FP_example.uf.union(obs[0], obs[i]) # Use the UnionFind class to accomplish the join
print('The joined groups of observations now looks like: \n', FP_example.uf.parent)

The joined groups of observations now looks like: 
 {'13': '12', '12': '12', '16': '67', '67': '53', '60': '6', '6': '28', '7': '61', '61': '29', '15': '65', '65': '77', '22': '74', '74': '74', '82': '23', '23': '23', '57': '2', '2': '2', '3': '58', '58': '58', '59': '4', '5': '4', '4': '26', '10': '9', '9': '9', '14': '63', '63': '63', '75': '21', '21': '21', '56': '25', '25': '25', '28': '28', '29': '29', '35': '34', '34': '34', '64': '37', '37': '37', '38': '66', '66': '66', '81': '40', '40': '40', '27': '26', '26': '58', '31': '30', '30': '32', '32': '32', '62': '36', '36': '36', '39': '79', '79': '79', '47': '46', '46': '46', '50': '72', '72': '72', '73': '52', '52': '52', '41': '45', '45': '42', '42': '42', '51': '77', '77': '77', '78': '53', '53': '53', '70': '44', '44': '43', '43': '43', '48': '71', '71': '71', '49': '76', '76': '63', '80': '54', '54': '54'}


&nbsp; &nbsp; &nbsp; &nbsp; In steps 3 and 4, to join the observation numbers for a given fish, the script calls the `find` method from `UnionFind` to flatten the structure of observation number groups. In other words, `find` must be called to ensure every observation number for a given fish has the same parent. The script does this by storing all distinct observation numbers in a set, and calling the `find` method for each number in this set. Remember, the `find` method flattens the structure. Run the following code and examine the output:

In [ ]:
# Step 3: Get the distinct obs
distinct_obs = set() # Creates an empty set (a data structure with a unique constraint) where obs values will be accumulated
for obs, tag in FP_example.data: # Go through every observation number and fill distinct_obs will distinct obs values
  distinct_obs.add(obs)
print('The group of distinct observation numbers looks like: \n', distinct_obs)

# Step 4: form the groups
groups = defaultdict(set)# Empty dictionary with unique contraint where the groups from the final output file will be stored
                         # This dictionary has parents as keys and group members as values
for obs in distinct_obs: # Iterate over every distinct obs
  root = FP_example.uf.find(obs) # Find the parent of obs, and thus flatten the structure created by step 2
  groups[root].add(obs) # Add the ultimate parent (parent after flattening) to groups dictionary
print('The final groups look like: \n', groups)

The group of distinct observation numbers looks like: 
 {'36', '68', '17', '57', '2', '32', '60', '73', '79', '25', '61', '34', '48', '62', '21', '53', '71', '59', '39', '10', '70', '18', '76', '29', '23', '1', '56', '31', '58', '45', '30', '54', '24', '77', '3', '75', '35', '27', '14', '49', '20', '13', '66', '74', '15', '81', '67', '38', '37', '51', '69', '55', '82', '46', '5', '11', '42', '33', '8', '4', '28', '47', '80', '22', '63', '41', '65', '26', '78', '9', '19', '72', '7', '44', '64', '52', '50', '16', '12', '6', '40', '43'}
The final groups look like: 
 defaultdict(<class 'set'>, {'36': {'62', '36'}, '68': {'68'}, '17': {'17'}, '2': {'57', '2'}, '32': {'31', '30', '32'}, '28': {'60', '6', '28'}, '52': {'73', '52'}, '79': {'79', '39'}, '25': {'56', '25'}, '29': {'7', '61', '29'}, '34': {'35', '34'}, '71': {'48', '71'}, '21': {'75', '21'}, '53': {'16', '67', '78', '53'}, '58': {'27', '5', '3', '59', '26', '58', '4'}, '9': {'10', '9'}, '43': {'44', '43', '70'}, '18': {'18'}, '63

The final groups dictionary from the last code output is then written into an output csv file. Therefore, all that the remaining chunk of code for the `process_group` method does is output the data stored in `groups` to a csv table in the form that you see in the file `FishPythonOutputTable.csv`.

###**`has_support_for`**

The purpose of the `has_support_for` method is self-explanatory in the doctsting and the comments provided in the script itself.